In [ ]:
import pandas as pd
import re
import io
from copy import deepcopy

In [ ]:
TEST = False

In [ ]:
if TEST:
    filename_a = "data/input_5a_test"
    filename_b = "data/input_5b_test"
    usecols = list(range(3))
else:
    filename_a = "data/input_5a"
    filename_b = "data/input_5b"
    usecols = list(range(9))

Read in container map from a utf-16 encoded text tile (saved via Excel) and parse into a DataFrame

In [ ]:
with open(filename_a, encoding='utf-16') as file:
    input_str_a = file.read()

In [ ]:
container_map = (pd
                 .read_csv( # Parse input as csv
                     io.StringIO( # Read string as input
                         '\n' # Join back into string with lines
                         .join(
                             [re.sub(r"    ", " .", # deal with blank entries by replacing with .
                                     re.sub(r"^   ","." ,line)) # If first column is blank replace with .
                              for line in input_str_a.split('\n')[:-1]] # Iteate lines (apart from last one)
                         )
                     ), 
                     sep=" ", 
                     header=None, 
                     usecols=usecols
                 )
                )
container_map.columns = list(map(lambda i: i+1, container_map.columns))
container_map = container_map.replace(".","")

In [ ]:
container_map

Read in list of instructions

In [ ]:
with open(filename_b) as file:
    input_str_b = file.read()

In [ ]:
instructions_str = input_str_b.strip("\n").split("\n")

In [ ]:
instructions = [re.findall(r'\d+', instruction) for instruction in instructions_str]
instructions[:4]

Define Stack class

In [ ]:
class Stack:
    
    content = ''
    
    def __init__(self, content):
        self.content = content
    
    def __repr__(self):
        return self.content
    
    def __str__(self):
        return self.content
    
    def add(self,containers,crane_model=9000):
        ''' Adds containers from list, in order based on crane model
        
        Args:
            containers (str): list of containers to be added
            crane_model (int): accepted values:[9000, 9001], defines whether
                               containers are added in reverse order or not
        '''
        if len(containers)>0:
            if crane_model==9000:
                self.content += containers[::-1]
            elif crane_model==9001:
                self.content += containers
            else:
                raise ValueError("Unrecognised Crane Model: {0}".format(crane_model))
        else:
            raise ValueError("Moving no containers!")
            
    def remove(self,n):
        '''Removes a number (n) of containers from the top of the stack'''
        if (n>0) & (n<=len(self.content)):
            containers = self.content[-n:]
            self.content = self.content[:len(self.content)-n]
            return containers
        elif n<0:
            raise ValueError("Moving no containers")
        elif n>len(self.content):
            raise ValueError("Moving more containers than on stack!")
    

Define helper functions which can convert between a container map in the form of a pandas DataFrame for visualisation and a list of stack object for executing crane instructions

In [ ]:
def map_to_stacks(container_map):
    '''Turn container map (DataFrame) into a list of stack objects'''
    stacks = [Stack(re.sub(r'[\W_]', 
                           '', 
                           ''
                           .join(list(container_map[i].values)))
                    .strip()
                    [::-1]) 
              for i in range(1,max(usecols)+2)]
    return stacks

In [ ]:
def stacks_to_map(stacks):
    '''Turn a list of stack objects into a container map'''
    container_map = (pd.DataFrame([['[{0}]'.format(container) 
                                    for container in list(stack.content)] 
                                   for stack in stacks])
                     .fillna('')
                     .transpose()
                     [::-1])
    container_map.columns = list(map(lambda i: i+1, container_map.columns))
    return container_map

In [ ]:
starting_stacks = map_to_stacks(container_map)

In [ ]:
starting_stacks

In [ ]:
starting_stacks[0]

In [ ]:
type(starting_stacks[0])

In [ ]:
stacks_to_map(starting_stacks)

Part 1 - Crane model 9000

In [ ]:
def one_move(instruction, stacks, verbose=False, crane_model=9000):
    '''Executes one crane instruction on a list of stacks
    Args:
        instruction (list(str)): List of numbers (as str) in the instruction
        stacks (list(Stack)): List of container stacks available
        verbose (Boolean): whether to print a map after each instruction
        crane_model (int): accepted values:[9000,9001], defines whether containers
                       are moved in reverse order or not.
                       
    Returns:
        list(Stacks): list of stacks after move has been executed
    '''
    n_containers, source , destination = list(map(int, instruction))
    if verbose: print("\nMove {0} from {1} to {2}\n".format(n_containers, source, destination))
    stacks[destination-1].add(stacks[source-1].remove(n_containers), crane_model=crane_model)
    if verbose: print(stacks_to_map(stacks))
    return stacks

In [ ]:
stacks = deepcopy(starting_stacks)
print(stacks_to_map(stacks))
for instruction in instructions:
    stacks = one_move(instruction, stacks)

In [ ]:
print(stacks_to_map(stacks))

In [ ]:
print(stacks)

In [ ]:
top_containers = ''
for stack in stacks:
    try:
        top_containers += stack.content[-1]
    except IndexError:
        continue

In [ ]:
TEST_ANSWER = 'CMZ'

In [ ]:
if TEST:
    assert top_containers == TEST_ANSWER
else: 
    print("Top containers are {0}".format(top_containers))

Part 2 - Crane model 9001

In [ ]:
starting_stacks

In [ ]:
stacks = deepcopy(starting_stacks)
print(stacks_to_map(stacks))
for instruction in instructions:
    stacks = one_move(instruction, stacks, crane_model=9001)

In [ ]:
print(stacks_to_map(stacks))

In [ ]:
top_containers = ''
for stack in stacks:
    try:
        top_containers += stack.content[-1]
    except IndexError:
        continue

In [ ]:
TEST_ANSWER = 'MCD'

In [ ]:
if TEST:
    assert top_containers == TEST_ANSWER
else: 
    print("Top containers are {0}".format(top_containers))